In [ ]:
# nltk.download()

In [48]:
from bs4 import BeautifulSoup
import csv
import datetime
import gensim
from gensim import corpora, models
import nltk
from nltk.corpus import twitter_samples
from nltk.book import *
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
import pickle
import random
import re
from time import sleep
from urllib.request import Request, urlopen

In [78]:
# url = "http://frontierdispatches.tumblr.com"
# url = "https://www.workshopdigital.com"
# url = "https://www.teamcolab.com"
url = "https://www.unboxedtechnology.com"
# url = "http://connorriley.me"
pages = set()
pages.add(url)
corpus = []
pages_corpus = []

# Scrape Web Page

In [23]:
def write_to_csv(row):
    writer.writerow(row)

In [79]:
def getLinks(pageUrl):
    global pages, url
#     html = urlopen(pageUrl)
    req = Request(pageUrl, headers={'User-Agent': 'Mozilla/5.0'})
    html = urlopen(req).read()
    bsObj = BeautifulSoup(html, "html.parser")
#     print(bsObj)
    page = []
    page.append(pageUrl)
#     try:
#         page.append(bsObj.h1.get_text().replace('\n',''))
#     except AttributeError:
#         print("This page is missing H1! No worries though!")
#     try:
#         page.append(bsObj.h2.get_text().replace('\n',''))
#     except AttributeError:
#         print("This page is missing H2! No worries though!")
#     try:
#         page.append(bsObj.h3.get_text().replace('\n',''))
#     except AttributeError:
#         print("This page is missing H3! No worries though!")
    try:
        p_list = bsObj.find(class_ ="container").findAll("p")#.find("main").findAll("p")
#         p_list = bsObj.find(class_ = "the-posts").findAll("p")#.find("main").findAll("p")
#         print(p_list)
        for p in p_list:
            p_edit = p.get_text(separator=' ').replace('\n','')
            words_filtered = [e.lower() for e in p_edit.split() if len(e) >= 3]
#             print(words_filtered)
            page.append(p_edit)
            for word in words_filtered:
                corpus.append(word)
    except AttributeError:
        print("This page is missing P! No worries though!")

    write_to_csv(page)    
    #absolute links
    for link in bsObj.findAll("a", href=re.compile("^("+url+")")):
    #relative links
#     for link in bsObj.find_all('a', href=re.compile(r'^(?!(?:[a-zA-Z][a-zA-Z0-9+.-]*:|//))')):
#         print(link.get('href'))
        print(link.attrs['href'])
        if 'href' in link.attrs:
            newPage = link.attrs['href']
            if newPage not in pages and '/about-us/' in newPage:# and '/wp-admin/' not in newPage:
                #We have encountered a new page
                print("----------------\n"+newPage)
                pages.add(newPage)
                sleep(random.uniform(1,5))
                try:
                    getLinks(url + newPage)
                except Exception as e:
                    print(e)
                continue
    

In [80]:
csvFile = open(url.split("://")[1]+".csv", 'w+', newline='')  
writer = csv.writer(csvFile)

In [81]:
getLinks(url)
csvFile.close()

https://www.unboxedtechnology.com/custom-training/
https://www.unboxedtechnology.com/custom-training/
https://www.unboxedtechnology.com/custom-training/onboarding-training/
https://www.unboxedtechnology.com/custom-training/product-training/
https://www.unboxedtechnology.com/custom-training/custom-sales-training/
https://www.unboxedtechnology.com/custom-training/leadership-training/
https://www.unboxedtechnology.com/products/spoke/
https://www.unboxedtechnology.com/custom-training/elearning-solutions/
https://www.unboxedtechnology.com/custom-training/interactive-learning-guides/
https://www.unboxedtechnology.com/custom-training/instructor-led/
https://www.unboxedtechnology.com/products/advisor/
https://www.unboxedtechnology.com/results/
https://www.unboxedtechnology.com/results/
https://www.unboxedtechnology.com/results/success-stories/
https://www.unboxedtechnology.com/results/testimonials/
https://www.unboxedtechnology.com/results/awards/
https://www.unboxedtechnology.com/about-us/
--

KeyboardInterrupt: 

# Stop Words

In [52]:
stop = set(stopwords.words('english'))
# print(stop)

In [82]:
stopped_tokens = [i for i in corpus if not i in stop]
# print(stopped_tokens)

# Begin Sentiment Analysis

In [54]:
f = open('sent_classifier.pickle', 'rb')
classifier = pickle.load(f)
f.close()

In [55]:
pos_strings = twitter_samples.strings('positive_tweets.json')
neg_strings = twitter_samples.strings('negative_tweets.json')

In [56]:
tweets = []
for words in pos_strings:
    words_filtered = [e.lower() for e in words.split() if len(e) >= 3]
    tweets.append((words_filtered, 'positive'))
for words in neg_strings:
    words_filtered = [e.lower() for e in words.split() if len(e) >= 3]
    tweets.append((words_filtered, 'negative'))

In [57]:
def get_words_in_doc(doc):
    all_words = []
    for (words, sentiment) in doc:
        all_words.extend(words)
    return all_words

def get_word_features(wordlist):
    wordlist = nltk.FreqDist(wordlist)
    word_features = wordlist.keys()
    return word_features

In [58]:
word_features = get_word_features(get_words_in_doc(tweets))

In [59]:
def extract_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in document_words)
    return features

In [83]:
print( classifier.prob_classify(extract_features(stopped_tokens)).max() )
print( classifier.prob_classify(extract_features(stopped_tokens))._prob_dict['positive'] )
print( classifier.prob_classify(extract_features(stopped_tokens))._prob_dict['negative'] )
print( classifier.prob_classify(extract_features(stopped_tokens)).prob('positive') )
print( classifier.prob_classify(extract_features(stopped_tokens)).prob('negative') )

negative
-1.7254781838321094
-0.5195244878439738
0.3023982752471159
0.6976017247528836


In [84]:
fdist1 = FreqDist(stopped_tokens)
print(fdist1.most_common(50))

[('uncomplicated,', 1), ('meet', 1), ('custom', 1), ('enablement', 1), ('solutions', 1), ('unrivaled,', 1), ('unboxed.', 1), ('unboxed', 1), ('minutes', 1), ('sales', 1), ('training', 1), ('unconventional,', 1)]


In [62]:
## creates id for each distinct word and assigns an ID
dictionary = corpora.Dictionary([stopped_tokens])

In [63]:
##turns the dictionary into a bag of words returning ID and frequency pairs
bag_of_words = [dictionary.doc2bow([text]) for text in stopped_tokens]

In [85]:
#  Parameters:

#  num_topics: required. An LDA model requires the user to determine how many topics should be generated. Our document set is small, so we’re only asking for three topics.
#  id2word: required. The LdaModel class requires our previous dictionary to map ids to strings.
#  passes: optional. The number of laps the model will take through corpus. The greater the number of passes, the more accurate the model will be. A lot of passes can be slow on a very large corpus.

ldamodel = gensim.models.ldamodel.LdaModel(bag_of_words, num_topics=5, id2word = dictionary, passes=30)

In [65]:
f = open(url.split("://")[1]+'_lda_model.pickle', 'wb')
pickle.dump(ldamodel, f)
f.close()

In [66]:
print(ldamodel.print_topics(num_topics=5, num_words=6))

[(0, '0.022*"design" + 0.018*"services" + 0.017*"also" + 0.015*"help" + 0.013*"information" + 0.013*"use"'), (1, '0.033*"co+lab" + 0.021*"one" + 0.019*"bullhorn" + 0.014*"android" + 0.013*"would" + 0.011*"ios"'), (2, '0.024*"plugin" + 0.022*"website" + 0.016*"work" + 0.015*"offer" + 0.013*"custom" + 0.012*"business."'), (3, '0.019*"wordpress" + 0.017*"learn" + 0.013*"new" + 0.011*"data" + 0.010*"team" + 0.009*"browser"'), (4, '0.031*"site" + 0.011*"need" + 0.011*"mobile" + 0.009*"using" + 0.009*"number" + 0.009*"project"')]


In [ ]:
# page = 'https://www.workshopdigital.com/blog/keyword-planner-vs-moz-keyword-explorer-vs-ahrefs-keywords-explorer/'
page = 'https://www.workshopdigital.com/about/team/Derek-Gleason/'
if '/blog/' not in page and '/case-studies/' not in page:
    print('ran')
else:
    print('skip')